In [136]:
import argparse
import glob
import numpy as np
import os
import torch

from torchvision import transforms
from torch.utils import data

import opt

from lib.image import unnormalize
from lib.img_io import load_image, writeLDR, writeEXR
from lib.io import load_ckpt
from lib.io import print_
from lib.util import make_dirs
from lib.util import get_saturated_regions
from network.softconvmask import SoftConvNotLearnedMaskUNet

In [137]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# Image load routines

Define a function for loading a single image. The function takes an image directory and an optional transformation to be applied to the loaded image and returns the transformed image and the corresponding saturation mask.

In [138]:
def load_image_tensor(input_dir, image_transform=None):
    # load image
    image = load_image(input_dir)

    # get saturation mask
    conv_mask = 1 - get_saturated_regions(image)
    conv_mask = torch.from_numpy(conv_mask).permute(2,0,1)

    # apply transform to input image
    if image_transform is not None:
        image = img_transform(image)

    return image, conv_mask

In [139]:
img_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=opt.MEAN, std=opt.STD)])

# Set dirs

In [141]:
# Path to the pre-trained weights.
WEIGHTS_DIR = './data/ldr2hdr.pth'

# Path to the image for inference.
IMAGE_DIR = './data/0014.png'

# Path to output directory where the final results will be saved.
OUTPUT_DIR = './'

# Name of the output image.
OUTPUT_IMAGE_NAME = '0014'

# Load Model

In [142]:
model = SoftConvNotLearnedMaskUNet().to(device)
model.print_network()
load_ckpt(WEIGHTS_DIR, [('model', model)])
model.eval()

	Network [SoftConvNotLearnedMaskUNet] was created. Total number of parameters: 51.5 million. To see the architecture, do print(network).


# Evaluate Model

## Load Image

In [143]:
image, mask = load_image_tensor(IMAGE_DIR, img_transform)
image = image[None, ...]
mask = mask[None, ...]

In [144]:
print("Image Saturation: %0.2f%%" % (100.0*(1-mask.mean().item())))

Image Saturation: 35.67%


In [145]:
image.shape

torch.Size([1, 3, 512, 1024])

## Model Inference

In [146]:
with torch.no_grad():
    pred_img = model(image.to(device), mask.to(device))

## Obtain final image

Compute the final HDR image by combining the well-exposed content of the input image (in the linear domain) and the output of the network in the saturated areas (eq. 1). To convert the input image to the linear domain, we use a gamma function to approximate the inverse of camera curve.   

In [147]:
# Convert pytorch tensors to numpy.
image = unnormalize(image).permute(0,2,3,1).numpy()[0,:,:,:]
mask = mask.permute(0,2,3,1).numpy()[0,:,:,:]
pred_img = pred_img.cpu().permute(0,2,3,1).numpy()[0,:,:,:]

# Convert the predicted image from log to the linear domain.
y_predict = np.exp(pred_img)-1

# Transforms the input image to the linear domain by using a gamma
# approximate to the inverse of camera curve. You can change this function
# with a more accurate representation of the camera curve.
gamma = np.power(image, 2)

# Compute the final HDR image.
H = mask*gamma + (1-mask)*y_predict

# Save images

Save final image and the intermediare images for debugging purpose.

In [149]:
writeLDR(mask, '{}/{}_mask.png'.format(OUTPUT_DIR, OUTPUT_IMAGE_NAME))
writeEXR(H, '{}/{}.exr'.format(OUTPUT_DIR, OUTPUT_IMAGE_NAME))
writeEXR(gamma, '{}/{}_gamma.exr'.format(OUTPUT_DIR, OUTPUT_IMAGE_NAME))